In [1]:
# import usual libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import f1_score

# our system imports
from aideme import *

%matplotlib inline

In [2]:
def plot_fscore(metrics):
    df_list = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics]
    avg = sum([df['fscore'][~df['fscore'].isna()] for df in df_list]) / len(df_list)
    avg.plot(ylim=[0,1], marker='o')
    plt.show()

In [3]:
# DUMMY DATA
N = int(1e5)
dim = 2
limit = 2 * (0.001)**(1. / dim)
rng = np.random.RandomState(0)
X = rng.uniform(low=-2, high=2, size=(N, dim))  # do not forget to standardize the data. For this distribution, it should be fine without it.
y_subspace = np.vstack([np.abs(X[:, i]) < limit for i in range(dim)]).T.astype('float')  # partial labels (for each subspace)
y = y_subspace.min(axis=1)

index = -10 * np.arange(len(X))
labeled_set = LabeledSet(y, y_subspace, index)

# visualize data distribution
print('selectivity :', 100 * y.sum() / len(y), '%')

#plt.figure(figsize=(10,8))
#plt.scatter(X[:, 0], X[:, 1], s=0.5, c=['b' if lb else 'r' for lb in y])
#plt.show()

selectivity : 0.105 %


# NO FACTORIZATION EXAMPLE

In [10]:
# SET-UP EXPLORATION CONFIGURATION
REPEAT = 1
NUMBER_OF_ITERATIONS = 100  # number of points to be labeled by the user

SUBSAMPLING = None  # 

INITIAL_SAMPLER = stratified_sampler(labeled_set, pos=1, neg=1)  # start with one random positive sample and one random negative sample
#INITIAL_SAMPLER = random_sampler(10)

CALLBACK = [ # callback functions to be called at the end of each iteration
    classification_metrics(labeled_set.labels, 'fscore'), 
    three_set_metric,
]
CALLBACK_SKIP = 10
PRINT_CALLBACK_RESULT = True

CONVERGENCE_CRITERIA = [
    max_iter_reached(NUMBER_OF_ITERATIONS),
    #all_points_are_known,
    #metric_reached_threshold('fscore', 0.8),
    #metric_reached_threshold('tsm', 0.9),
]

NOISE_INJECTOR = random_noise_injector(0.05)  # None, random_noise_injector, gaussian_noise_injector

SEED = list(range(REPEAT))

explore = PoolBasedExploration(INITIAL_SAMPLER, SUBSAMPLING, CALLBACK, CALLBACK_SKIP, PRINT_CALLBACK_RESULT, CONVERGENCE_CRITERIA, NOISE_INJECTOR)

# ACTIVE LEARNING ALGORITHMS
#learner = RandomSampler(SVC(C=1e5, kernel='rbf', gamma='auto'))  # choose a random point
learner = SimpleMargin(C=1e7, kernel='rbf')  # choose point closest to SVM decision boundary
#learner = DualSpaceModel(learner, sample_unknown_proba=0.5, mode='positive')  # Dual Space model
#learner = KernelQueryByCommittee(kernel='rbf', sampling='deterministic', n_samples=8, warmup=1000, thin=100, rounding=True)  # version space algorithm

# FACTORIZED ALGORITHMS
PARTITION = [[0], [1]]
#learner = FactorizedDualSpaceModel(SimpleMargin(C=1024, kernel='rbf'), partition=PARTITION, mode='positive', sample_unknown_proba=0.5)  # Dual Space model
#learner = SubspatialVersionSpace(warmup=100, thin=10, n_samples=8, rounding=True, kernel='rbf', gamma=None, partition=PARTITION, label_function='AND', loss='GREEDY')
#learner = SubspatialSimpleMargin(C=1024, kernel='rbf', gamma=5, partition=PARTITION, label_function='AND')


# RUN EXPLORATION
metrics = explore.run(X, labeled_set, learner, repeat=REPEAT, seeds=SEED)  # 'repeat' specifies how many times to repeat the exploration process

# COMPUTE AVERAGE F-SCORE OVER ALL REPEATS AND PLOT
#plot_fscore(metrics)
df = pd.DataFrame.from_dict({i: metric for i, metric in enumerate(metrics[0])}, orient='index')
df

iter: 1, fscore: 0.002917841908546498
iter: 11, fscore: 0.01289371891692761
iter: 21, fscore: 0.02945195081220586
iter: 31, fscore: 0.01143379830779785
iter: 41, fscore: 0.006707946336429308
iter: 51, fscore: 0.022896698615548456
iter: 61, fscore: 0.016951672862453533
iter: 71, fscore: 0.019661952397378408
iter: 81, fscore: 0.020761245674740483
iter: 91, fscore: 0.021623913504345983
iter: 101, fscore: 0.024161073825503355


,phase,iter_time,labeled_indexes,final_labels,partial_labels,fit_time,get_next_time,fscore,callback_time
0,begin,0.002019,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,initial_sampling,0.035486,"[-966950, -231540]","[1.0, 0.0]","[[1.0, 1.0], [0.0, 1.0]]",0.000846,0.034640,0.002918,0.443287
2,exploration,0.035010,[-266110],[0.0],"[[0.0, 0.0]]",0.000634,0.034376,NaN,NaN
3,exploration,0.036457,[-948690],[0.0],"[[0.0, 0.0]]",0.000411,0.036046,NaN,NaN
4,exploration,0.036928,[-665530],[1.0],"[[1.0, 0.0]]",0.000488,0.036440,NaN,NaN
...,...,...,...,...,...,...,...,...,...
97,exploration,0.264160,[-505640],[0.0],"[[0.0, 0.0]]",0.215023,0.049137,NaN,NaN
98,exploration,0.222596,[-393060],[0.0],"[[0.0, 0.0]]",0.170575,0.052022,NaN,NaN
99,exploration,0.261278,[-656140],[1.0],"[[0.0, 1.0]]",0.212295,0.048982,NaN,NaN
100,exploration,0.198504,[-120020],[0.0],"[[1.0, 0.0]]",0.147289,0.051215,NaN,NaN
